In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

pivot_df = pd.read_csv('pivot_df.csv')
study_plan=pd.read_csv('study_plan_cleaned.csv')




In [2]:
pivot_df

,STD_NO,111023,111033,111512,112063,112123,112532,113133,113233,113353,...,313113,313513,313523,313613,31373,gender_encoded,tawjihi_country_encoded,TAWJIH_GPA,tawjihi_branch_encoded,Cluster
0,313134,35.0,0.0,0.0,35.0,0.0,50.0,0.0,0.0,0.0,...,0.0,0.0,43.0,0.0,0.0,1.0,2.0,86.0,0.0,1
1,318942,0.0,0.0,0.0,51.0,50.0,51.0,50.0,55.0,55.0,...,77.0,0.0,0.0,0.0,35.0,1.0,3.0,72.0,5.0,0
2,324629,54.0,70.0,80.0,61.0,54.0,62.0,52.0,65.0,50.0,...,76.0,0.0,0.0,0.0,0.0,1.0,2.0,87.0,4.0,0
3,325080,54.0,51.0,69.0,70.0,68.0,98.0,0.0,72.0,60.0,...,78.0,0.0,0.0,67.0,0.0,1.0,2.0,81.0,1.0,0
4,325201,99.0,100.0,95.0,84.0,72.0,100.0,83.0,84.0,85.0,...,0.0,0.0,0.0,0.0,35.0,1.0,2.0,95.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,381169,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,84.0,10.0,1
315,381290,75.0,59.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,71.0,8.0,1
316,381312,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,64.0,8.0,1
317,381873,58.0,35.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,87.0,0.0,1


In [3]:
# Data preprocessing

# Replacing 0s with NaN in the grade columns only
#the grade columns start from the second column and end before the last six columns
pivot_df.iloc[:, 1:-6] = pivot_df.iloc[:, 1:-6].applymap(lambda x: pd.NA if x == 0 else x)

# normalizing the grades (35 is the minimum grade)
min_grade = 35
pivot_df.iloc[:, 1:-6] = pivot_df.iloc[:, 1:-6].apply(lambda x: x - min_grade)

# normalizing the TAWJIH_GPA column (60 is the min)
pivot_df['TAWJIH_GPA_normalized'] = (pivot_df['TAWJIH_GPA'] - 60) / (100 - 60)

pivot_df_filled = pivot_df.copy()

# Replace NaNs with 0s only in the grade columns for similarity computation
pivot_df_filled.iloc[:, 1:-6] = pivot_df_filled.iloc[:, 1:-6].fillna(0)


C:\Users\User\AppData\Local\Temp\ipykernel_14160\1247051914.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  pivot_df_filled.iloc[:, 1:-6] = pivot_df_filled.iloc[:, 1:-6].fillna(0)


In [4]:
pivot_df_filled

,STD_NO,111023,111033,111512,112063,112123,112532,113133,113233,113353,...,313513,313523,313613,31373,gender_encoded,tawjihi_country_encoded,TAWJIH_GPA,tawjihi_branch_encoded,Cluster,TAWJIH_GPA_normalized
0,313134,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,1.0,2.0,86.0,0.0,1,0.650
1,318942,0.0,0.0,0.0,16.0,15.0,16.0,15.0,20.0,20.0,...,0.0,0.0,0.0,35.0,1.0,3.0,72.0,5.0,0,0.300
2,324629,19.0,35.0,45.0,26.0,19.0,27.0,17.0,30.0,15.0,...,0.0,0.0,0.0,0.0,1.0,2.0,87.0,4.0,0,0.675
3,325080,19.0,16.0,34.0,35.0,33.0,63.0,0.0,37.0,25.0,...,0.0,0.0,32.0,0.0,1.0,2.0,81.0,1.0,0,0.525
4,325201,64.0,65.0,60.0,49.0,37.0,65.0,48.0,49.0,50.0,...,0.0,0.0,0.0,35.0,1.0,2.0,95.0,1.0,0,0.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,381169,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,84.0,10.0,1,0.600
315,381290,40.0,24.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,71.0,8.0,1,0.275
316,381312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,64.0,8.0,1,0.100
317,381873,23.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,87.0,0.0,1,0.675


In [5]:
#Checking which types corresponding to which labels  (for readability)
unique_pairs = set(zip(study_plan['REQ_TYP'], study_plan['REQ_TYPEE']))

for req_number, req_description in unique_pairs:
    print(f"Requirement Type: {req_number}, Description: {req_description}")

Requirement Type: 3, Description: Compulsory School Requirements 
Requirement Type: 5, Description: Elective Program Requirements
Requirement Type: 1, Description: Compulsory University Requirements
Requirement Type: 2, Description: Elective University Requirements 
Requirement Type: 4, Description: Compulsory Program Requirements


In [6]:
#Checking which types corresponding to which labels (for readability)
unique_pairs = set(zip(study_plan['RULE_ID'], study_plan['RULE_DESC']))


for req_number, req_description in unique_pairs:
    print(f"Requirement Type: {req_number}, Description: {req_description}")

Requirement Type: 3, Description: Finish set amount of hours
Requirement Type: 0, Description: 0
Requirement Type: 1, Description: Prerequisite
Requirement Type: 2, Description: Concurrent


In [7]:
study_plan

,COURSE_ID,COURSE_DESCE,COURSE_TYP,COURSE_TYPEE,LEVEL_ID,LEVEL_DESCE,REQ_TYP,REQ_TYPEE,MARK_TYP,MARK_TYPEE,WEB_FLAG,LOAD_HOURS,REQ_HOURS,PRE_COURSE_ID,RULE_ID,RULE_DESC
0,11253,Object Oriented Programming Lab,2,Lab,2,Second Year,4,Compulsory Program Requirements,1,Distributed 60/40,0,1,68,11206,2,Concurrent
1,11212,Data Structures and Introduction to Algorithms,1,lecture,2,Second Year,4,Compulsory Program Requirements,1,Distributed 60/40,0,3,68,11206,1,Prerequisite
2,11313,Algorithms Design and Analysis,1,lecture,3,Third Year,4,Compulsory Program Requirements,1,Distributed 60/40,0,3,68,11212,1,Prerequisite
3,11354,Database Systems Lab,2,Lab,3,Third Year,4,Compulsory Program Requirements,1,Distributed 60/40,0,1,68,11323,2,Concurrent
4,11355,Operating Systems Lab,2,Lab,3,Third Year,4,Compulsory Program Requirements,1,Distributed 60/40,0,1,68,11335,2,Concurrent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,14141,Introduction to Data Science lab,2,Lab,1,First Year,4,Compulsory Program Requirements,1,Distributed 60/40,1,1,68,14140,2,Concurrent
58,14331,Artificial Intelligence lab,2,Lab,1,First Year,4,Compulsory Program Requirements,1,Distributed 60/40,1,1,68,14330,2,Concurrent
59,14271,Statistics and Probability for Data Science lab,2,Lab,1,First Year,4,Compulsory Program Requirements,1,Distributed 60/40,1,1,68,14270,2,Concurrent
60,11323,Database Systems,1,lecture,3,Third Year,4,Compulsory Program Requirements,1,Distributed 60/40,0,3,68,11212,1,Prerequisite


In [8]:
# columns to be dropped
columns_to_drop = ['COURSE_TYP', 'COURSE_TYPEE', 'LEVEL_ID', 'LEVEL_DESCE', 
                   'MARK_TYP', 'MARK_TYPEE', 'WEB_FLAG', 'LOAD_HOURS', 'REQ_HOURS','REQ_TYPEE','RULE_DESC']

study_plan = study_plan.drop(columns=columns_to_drop, axis=1)


In [9]:
# Changing rules for courses that need new prerequisites
updates = {
    14364: {'new_rule_id': 1, 'new_pre_course_id': 14270},
    14452: {'new_rule_id': 1, 'new_pre_course_id': 14350}
}

# Loop through the dictionary and update study_plan
for course_id, update_info in updates.items():
    study_plan.loc[study_plan['COURSE_ID'] == course_id, 'RULE_ID'] = update_info['new_rule_id']
    study_plan.loc[study_plan['COURSE_ID'] == course_id, 'PRE_COURSE_ID'] = update_info['new_pre_course_id']

In [10]:
study_plan['PRE_COURSE_ID'] = study_plan['PRE_COURSE_ID'].fillna(0).astype(int)


In [11]:
study_plan

,COURSE_ID,COURSE_DESCE,REQ_TYP,PRE_COURSE_ID,RULE_ID
0,11253,Object Oriented Programming Lab,4,11206,2
1,11212,Data Structures and Introduction to Algorithms,4,11206,1
2,11313,Algorithms Design and Analysis,4,11212,1
3,11354,Database Systems Lab,4,11323,2
4,11355,Operating Systems Lab,4,11335,2
...,...,...,...,...,...
57,14141,Introduction to Data Science lab,4,14140,2
58,14331,Artificial Intelligence lab,4,14330,2
59,14271,Statistics and Probability for Data Science lab,4,14270,2
60,11323,Database Systems,4,11212,1


In [12]:

print(study_plan.dtypes)


COURSE_ID         int64
COURSE_DESCE     object
REQ_TYP           int64
PRE_COURSE_ID     int32
RULE_ID           int64
dtype: object


In [13]:
# cosine similarity
grades = pivot_df_filled.iloc[:, 1:].values
user_similarity = cosine_similarity(grades)

# Converting the similarity matrix to a df
user_similarity_df = pd.DataFrame(user_similarity, index=pivot_df_filled['STD_NO'], columns=pivot_df_filled['STD_NO'])


In [14]:
def check_course_rules(course_id, pivot_df, study_plan):
    course_info = study_plan[study_plan['COURSE_ID'] == course_id]
    if not course_info.empty:
        rule_id = course_info.iloc[0]['RULE_ID']
        pre_course_id = course_info.iloc[0]['PRE_COURSE_ID']

        if rule_id == 1:  # Prerequisite
            # Check if the student has completed the prerequisite
            if pre_course_id not in pivot_df.columns or pd.isna(pivot_df.at[ pre_course_id]):#user_id,
                return False  # Do not recommend if the prerequisite is not completed

        elif rule_id == 2:  # Concurrent
            # Recommend the concurrent course
            return pre_course_id

    return True  

In [15]:
# function to generate course recommendations based on a range of total hours
def generate_recommendations(user_id, user_similarity_df, pivot_df_filled, min_hours=9):
    if user_id not in user_similarity_df.index:
        return "User ID not found in the dataset."

    user_similarities = user_similarity_df.loc[user_id]
    similar_users = user_similarities.sort_values(ascending=False).index
    finished_courses = pivot_df_filled[pivot_df_filled['STD_NO'] == user_id].dropna(axis=1).columns.tolist()
    course_scores = {}

    for similar_user in similar_users:
        if similar_user != user_id:
            similar_user_courses = pivot_df_filled[pivot_df_filled['STD_NO'] == similar_user].dropna(axis=1)
            for course in similar_user_courses:
                if course not in finished_courses and course != 'STD_NO':
                    course_grade = similar_user_courses[course].values[0]
                    course_scores[course] = course_scores.get(course, 0) + course_grade

    sorted_courses = sorted(course_scores, key=course_scores.get, reverse=True)

    # Select courses based on total hours, aiming to stay within the min and max range with a tolerance of 2 
    recommended_courses = []
    total_hours = 0
    for course in sorted_courses:
        load_hour = int(str(course)[-1])
        course_id_int = int(str(course)[:-1])

        rule_check = check_course_rules(course_id_int, pivot_df_filled, study_plan)
        if rule_check is True or rule_check is not False:  # True or concurrent course ID
            # Add course to recommendations
            recommended_courses.append(course)
            total_hours += load_hour

            # If a concurrent course is returned, add it too
            if rule_check is not True:
                concurrent_course = str(rule_check) + str(load_hour)  # assuming same load hour
                if concurrent_course not in finished_courses:
                    recommended_courses.append(concurrent_course)
                    total_hours += load_hour

        if total_hours >= min_hours:
            break
    max_hours=min_hours+2
#    Check if the total hours are within the required range
    if total_hours < min_hours or total_hours > max_hours:
        return f"Unable to meet the hour requirement within 9-21 hours range."

    return recommended_courses


In [16]:
#array to store recommendations
all_recommendations = []

# Ask for user ID
user_input_id = input("Enter the Student ID: ")
try:
    user_id = int(user_input_id)
except ValueError:
    print("Invalid input. Please enter a valid Student ID.")
    # Handle invalid input as needed

# Ask for the number of hours needed
user_input_rec = input("Enter the number of hours you want: ")
try:
    target_hours = int(user_input_rec)
    result = generate_recommendations(user_id, user_similarity_df, pivot_df, target_hours)

    if isinstance(result, str):
        print(result)
    else:
        print(f"Recommended courses for user {user_id} within {target_hours} hours: {result}")
        # save the recommendations in the array
        all_recommendations.append((user_id, result))

except ValueError:
    print("Invalid input. Please enter a numerical value for the number of hours.")

Recommended courses for user 359312 within 11 hours: ['311303', '113542', '113232', '143312', '143302']


In [17]:
for student, courses in all_recommendations:
    print(f"Student Number: {student}")
    for course in courses:
        #the course ID is an integer followed by a single digit for load hour
        course_id_str = str(course).split('.')[0]  # Remove any decimal part
        course_id_int = int(course_id_str[:-1])    # Convert to integer, excluding load hour for readability

        course_desc = study_plan.loc[study_plan['COURSE_ID'] == course_id_int, 'COURSE_DESCE']
        if not course_desc.empty:
            print(f"Course ID: {course_id_int}, Description: {course_desc.iloc[0]}")
        else:
            print(f"Course ID: {course_id_int}, Description: Not found")


Student Number: 359312
Course ID: 31130, Description: Foreign languages
Course ID: 11354, Description: Database Systems Lab
Course ID: 11323, Description: Database Systems
Course ID: 14331, Description: Artificial Intelligence lab
Course ID: 14330, Description: Artificial Intelligence
